In [1]:
from fastai.vision.all import *

In [2]:
path=untar_data(URLs.PETS)/'images'

In [3]:
def is_cat(x):
  return x[0].isupper()

In [4]:
dls=ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42, label_func=is_cat, item_tfms=Resize(224)
)

In [6]:
learn=cnn_learner(dls, resnet34,  metrics=error_rate)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 136MB/s]


In [7]:
learn.fine_tune(1)

epoch,train_loss,valid_loss,error_rate,time
0,0.167164,0.024527,0.006766,00:49


epoch,train_loss,valid_loss,error_rate,time
0,0.057845,0.029944,0.006089,00:52
